In [1]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_dataset
import evaluate
import numpy as np
import torch

print("Imports OK !")

Imports OK !


In [2]:
# Charger le modèle sauvegardé
tokenizer = CamembertTokenizerFast.from_pretrained("./camembert-ner-final")
model = CamembertForTokenClassification.from_pretrained("./camembert-ner-final")

if torch.cuda.is_available():
    model = model.to("cuda")
    print("✓ Modèle chargé sur GPU !")

✓ Modèle chargé sur GPU !


In [3]:
# Charger et préparer le dataset test
dataset = load_dataset("Jean-Baptiste/wikiner_fr")
label_names = ['O', 'LOC', 'PER', 'MISC', 'ORG']

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
print("Dataset prêt !")

Map:   0%|          | 0/120682 [00:00<?, ? examples/s]

Map:   0%|          | 0/13410 [00:00<?, ? examples/s]

Dataset prêt !


In [5]:
# Évaluation finale
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    true_labels = []
    true_preds = []
    
    for pred, label in zip(predictions, labels):
        true_label = []
        true_pred = []
        for p, l in zip(pred, label):
            if l != -100:
                true_label.append(label_names[l])
                true_pred.append(label_names[p])
        true_labels.append(true_label)
        true_preds.append(true_pred)
    
    results = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./temp", per_device_eval_batch_size=64),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Évaluation sur test set
results = trainer.evaluate(tokenized_dataset["test"])

print("\n" + "="*50)
print("RÉSULTATS FINAUX NER - TEST SET")
print("="*50)
print(f"F1-Score  : {results['eval_f1']*100:.2f}%")
print(f"Precision : {results['eval_precision']*100:.2f}%")
print(f"Recall    : {results['eval_recall']*100:.2f}%")
print(f"Accuracy  : {results['eval_accuracy']*100:.2f}%")
print("="*50)

/tmp/ipykernel_209133/3179073934.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/home/camembert/.local/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/camembert/.local/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/camembert/.local/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/camembert/.local/lib/python3.12/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))



RÉSULTATS FINAUX NER - TEST SET
F1-Score  : 88.07%
Precision : 87.52%
Recall    : 88.62%
Accuracy  : 98.46%
